In [3]:
# Importing libraies into notebook

import pandas as pd
import numpy as np
import os
import seaborn as sns



In [4]:
# importing py files into notebook
import env

from env import host, user, password

import acquire

Acquire  Telco customer data from

In [5]:
df = acquire.get_telco_data()

Print Data frame information and get small look in to the data

In [6]:
# To Show all columns in dataframe
pd.set_option('display.max_columns', None)

In [7]:
# Show dataframe info and snapshot of dataframe#
print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


There are no null values. There are columns that have relationships, columns that multiple have variables that need to be embarked.

In [8]:
pd.crosstab(df['payment_type_id'], df['payment_type'])

payment_type,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
payment_type_id,,,,
1,0,0,2365,0
2,0,0,0,1612
3,1544,0,0,0
4,0,1522,0,0


In [9]:
pd.crosstab(df['internet_service_type_id'], df['internet_service_type'])

internet_service_type,DSL,Fiber optic,None
internet_service_type_id,,,
1,2421,0,0
2,0,3096,0
3,0,0,1526


In [10]:
pd.crosstab(df['contract_type_id'], df['contract_type'])

contract_type,Month-to-month,One year,Two year
contract_type_id,,,
1,3875,0,0
2,0,1473,0
3,0,0,1695


After crosstabbing the corresponding columns and reviewing the data. I will drop columns ' contract_type_id', 'internet_service_type_id', 'payment_type_id', and 'customer_id'

In [11]:
# prior to editing the data frame, I will save the raw dataframe to a function just in case I mess up while coding

telco_raw = acquire.get_telco_data()

In [12]:
# Drop columns
df = df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'])

In [13]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [14]:
df.shape

(7043, 20)

In [15]:
# remove any white space
df['total_charges'] = df['total_charges'].str.strip()

In [16]:
# verification white space is removed from total charges
df = df[df.total_charges != '']
df.shape

(7032, 20)

In [23]:
telco_raw[telco_raw.total_charges == ''].shape

(0, 24)

In [18]:
# Convert total charges data type to a float

df['total_charges'] = df.total_charges.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7032 non-null   object 
 1   senior_citizen         7032 non-null   int64  
 2   partner                7032 non-null   object 
 3   dependents             7032 non-null   object 
 4   tenure                 7032 non-null   int64  
 5   phone_service          7032 non-null   object 
 6   multiple_lines         7032 non-null   object 
 7   online_security        7032 non-null   object 
 8   online_backup          7032 non-null   object 
 9   device_protection      7032 non-null   object 
 10  tech_support           7032 non-null   object 
 11  streaming_tv           7032 non-null   object 
 12  streaming_movies       7032 non-null   object 
 13  paperless_billing      7032 non-null   object 
 14  monthly_charges        7032 non-null   float64
 15  tota

In [19]:
# printing value counts for each column
for c in df.columns:
    print ("---- %s ---" % c)
    print (df[c].value_counts())

---- gender ---
Male      3549
Female    3483
Name: gender, dtype: int64
---- senior_citizen ---
0    5890
1    1142
Name: senior_citizen, dtype: int64
---- partner ---
No     3639
Yes    3393
Name: partner, dtype: int64
---- dependents ---
No     4933
Yes    2099
Name: dependents, dtype: int64
---- tenure ---
1     613
72    362
2     238
3     200
4     176
     ... 
38     59
28     57
39     56
44     51
36     50
Name: tenure, Length: 72, dtype: int64
---- phone_service ---
Yes    6352
No      680
Name: phone_service, dtype: int64
---- multiple_lines ---
No                  3385
Yes                 2967
No phone service     680
Name: multiple_lines, dtype: int64
---- online_security ---
No                     3497
Yes                    2015
No internet service    1520
Name: online_security, dtype: int64
---- online_backup ---
No                     3087
Yes                    2425
No internet service    1520
Name: online_backup, dtype: int64
---- device_protection ---
No         

In [20]:
# encode the coresponding columns
df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})

In [21]:
print(df.shape)
df.head()

(7032, 26)


,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,gender_encoded,partner_encoded,dependents_encoded,phone_service_encoded,paperless_billing_encoded,churn_encoded
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check,1,1,1,1,1,0
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check,0,0,0,1,0,0
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check,0,0,0,1,1,1
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check,0,1,0,1,1,1
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check,1,1,0,1,1,1


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   gender                     7032 non-null   object 
 1   senior_citizen             7032 non-null   int64  
 2   partner                    7032 non-null   object 
 3   dependents                 7032 non-null   object 
 4   tenure                     7032 non-null   int64  
 5   phone_service              7032 non-null   object 
 6   multiple_lines             7032 non-null   object 
 7   online_security            7032 non-null   object 
 8   online_backup              7032 non-null   object 
 9   device_protection          7032 non-null   object 
 10  tech_support               7032 non-null   object 
 11  streaming_tv               7032 non-null   object 
 12  streaming_movies           7032 non-null   object 
 13  paperless_billing          7032 non-null   objec

In [24]:
# Create dummies for columns
dummy_df = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type']], dummy_na=False, \
                              drop_first=True)   

In [25]:
dummy_df

,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,online_security_Yes,online_backup_No internet service,online_backup_Yes,device_protection_No internet service,device_protection_Yes,tech_support_No internet service,tech_support_Yes,streaming_tv_No internet service,streaming_tv_Yes,streaming_movies_No internet service,streaming_movies_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
3,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0
4,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
7039,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
7040,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7041,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1


In [26]:
 # Merging dataframe with dummy dataframe
 df = pd.concat([df, dummy_df], axis=1)
 df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,gender_encoded,partner_encoded,dependents_encoded,phone_service_encoded,paperless_billing_encoded,churn_encoded,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,online_security_Yes,online_backup_No internet service,online_backup_Yes,device_protection_No internet service,device_protection_Yes,tech_support_No internet service,tech_support_Yes,streaming_tv_No internet service,streaming_tv_Yes,streaming_movies_No internet service,streaming_movies_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check,0,1,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1


In [27]:
df.shape

(7032, 47)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 47 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   gender                                 7032 non-null   object 
 1   senior_citizen                         7032 non-null   int64  
 2   partner                                7032 non-null   object 
 3   dependents                             7032 non-null   object 
 4   tenure                                 7032 non-null   int64  
 5   phone_service                          7032 non-null   object 
 6   multiple_lines                         7032 non-null   object 
 7   online_security                        7032 non-null   object 
 8   online_backup                          7032 non-null   object 
 9   device_protection                      7032 non-null   object 
 10  tech_support                           7032 non-null   object 
 11  stre

Verifiying the values match up after dummy and encoded values. FYI columns that created dummies dropped the first value. Dropped value will be 0.

In [30]:
pd.crosstab(df['gender'], df['gender_encoded'])

gender_encoded,0,1
gender,,
Female,0,3483
Male,3549,0


In [47]:
df.gender.value_counts()

Male      3549
Female    3483
Name: gender, dtype: int64

In [31]:
pd.crosstab(df['partner'], df['partner_encoded'])

partner_encoded,0,1
partner,,
No,3639,0
Yes,0,3393


In [48]:
df.partner.value_counts()

No     3639
Yes    3393
Name: partner, dtype: int64

In [32]:
pd.crosstab(df['dependents'], df['dependents_encoded'])

dependents_encoded,0,1
dependents,,
No,4933,0
Yes,0,2099


In [49]:
df.dependents.value_counts()

No     4933
Yes    2099
Name: dependents, dtype: int64

In [33]:
pd.crosstab(df['phone_service'], df['phone_service_encoded'])

phone_service_encoded,0,1
phone_service,,
No,680,0
Yes,0,6352


In [50]:
df.phone_service.value_counts()

Yes    6352
No      680
Name: phone_service, dtype: int64

In [34]:
pd.crosstab(df['paperless_billing'], df['paperless_billing_encoded'])

paperless_billing_encoded,0,1
paperless_billing,,
No,2864,0
Yes,0,4168


In [51]:
df.paperless_billing.value_counts()

Yes    4168
No     2864
Name: paperless_billing, dtype: int64

In [35]:
pd.crosstab(df['churn'], df['churn_encoded'])

churn_encoded,0,1
churn,,
No,5163,0
Yes,0,1869


In [52]:
df.churn.value_counts()

No     5163
Yes    1869
Name: churn, dtype: int64

In [36]:
df.groupby('multiple_lines')[['multiple_lines_No phone service', 'multiple_lines_Yes']].sum()

,multiple_lines_No phone service,multiple_lines_Yes
multiple_lines,,
No,0.0,0.0
No phone service,680.0,0.0
Yes,0.0,2967.0


In [53]:
df.multiple_lines.value_counts()

No                  3385
Yes                 2967
No phone service     680
Name: multiple_lines, dtype: int64

In [54]:
df.groupby('online_security')[['online_security_No internet service', 'online_security_Yes']].sum()

,online_security_No internet service,online_security_Yes
online_security,,
No,0.0,0.0
No internet service,1520.0,0.0
Yes,0.0,2015.0


In [55]:
df.online_security.value_counts()

No                     3497
Yes                    2015
No internet service    1520
Name: online_security, dtype: int64

In [38]:
df.groupby('online_backup')[['online_backup_No internet service', 'online_backup_Yes']].sum()

,online_backup_No internet service,online_backup_Yes
online_backup,,
No,0.0,0.0
No internet service,1520.0,0.0
Yes,0.0,2425.0


In [56]:
df.online_backup.value_counts()

No                     3087
Yes                    2425
No internet service    1520
Name: online_backup, dtype: int64

In [39]:
   df.groupby('device_protection')[['device_protection_No internet service', 'device_protection_Yes']].sum()

,device_protection_No internet service,device_protection_Yes
device_protection,,
No,0.0,0.0
No internet service,1520.0,0.0
Yes,0.0,2418.0


In [57]:
df.device_protection.value_counts()

No                     3094
Yes                    2418
No internet service    1520
Name: device_protection, dtype: int64

In [40]:
   df.groupby('tech_support')[['tech_support_No internet service', 'tech_support_Yes']].sum()

,tech_support_No internet service,tech_support_Yes
tech_support,,
No,0.0,0.0
No internet service,1520.0,0.0
Yes,0.0,2040.0


In [58]:
df.tech_support.value_counts()

No                     3472
Yes                    2040
No internet service    1520
Name: tech_support, dtype: int64

In [41]:
   df.groupby('streaming_tv')[['streaming_tv_No internet service', 'streaming_tv_Yes']].sum()

,streaming_tv_No internet service,streaming_tv_Yes
streaming_tv,,
No,0.0,0.0
No internet service,1520.0,0.0
Yes,0.0,2703.0


In [59]:
df.streaming_tv.value_counts()

No                     2809
Yes                    2703
No internet service    1520
Name: streaming_tv, dtype: int64

In [42]:
   df.groupby('streaming_movies')[['streaming_movies_No internet service', 'streaming_movies_Yes']].sum()

,streaming_movies_No internet service,streaming_movies_Yes
streaming_movies,,
No,0.0,0.0
No internet service,1520.0,0.0
Yes,0.0,2731.0


In [60]:
df.streaming_movies.value_counts()

No                     2781
Yes                    2731
No internet service    1520
Name: streaming_movies, dtype: int64

In [43]:
   df.groupby('contract_type')[['contract_type_One year', 'contract_type_Two year']].sum()

,contract_type_One year,contract_type_Two year
contract_type,,
Month-to-month,0.0,0.0
One year,1472.0,0.0
Two year,0.0,1685.0


In [61]:
df.contract_type.value_counts()

Month-to-month    3875
Two year          1685
One year          1472
Name: contract_type, dtype: int64

In [45]:
df.groupby('internet_service_type')[['internet_service_type_Fiber optic', 'internet_service_type_None']].sum()


,internet_service_type_Fiber optic,internet_service_type_None
internet_service_type,,
DSL,0.0,0.0
Fiber optic,3096.0,0.0
None,0.0,1520.0


In [62]:
df.internet_service_type.value_counts()

Fiber optic    3096
DSL            2416
None           1520
Name: internet_service_type, dtype: int64

In [46]:
df.groupby('payment_type')[['payment_type_Credit card (automatic)', 'payment_type_Electronic check', 'payment_type_Mailed check']].sum()


,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
payment_type,,,
Bank transfer (automatic),0.0,0.0,0.0
Credit card (automatic),1521.0,0.0,0.0
Electronic check,0.0,2365.0,0.0
Mailed check,0.0,0.0,1604.0


In [63]:
df.payment_type.value_counts()

Electronic check             2365
Mailed check                 1604
Bank transfer (automatic)    1542
Credit card (automatic)      1521
Name: payment_type, dtype: int64

In [66]:
def prep_telco():
    df = acquire.get_telco_data()
    df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], inplace=True)
    df['total_charges'] = df['total_charges'].str.strip()
    df = df[df.total_charges != '']
    df['total_charges'] = df.total_charges.astype(float)
    df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})
    dummy_df = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type']], dummy_na=False, \
                              drop_first=True)   
    df = pd.concat([df, dummy_df], axis=1)
    return df

Testing testing testing testing testing

In [64]:
from sklearn.model_selection import train_test_split
seed = 123

In [65]:
telco_train, telco_test = train_test_split(df, test_size=.2, random_state=123, stratify=df.churn )
telco_train, telco_validate = train_test_split(telco_train, test_size=.3, random_state=123, stratify=telco_train.churn)

print(telco_train.shape)
print(telco_validate.shape)
print(telco_test.shape)

(3937, 47)
(1688, 47)
(1407, 47)
